# Lab 7: Mining Social-Network Graphs
Data Mining 2019/2020 <br> 
Ruben Wiersma and Gosia Migut

**WHAT** This *optional* lab consists of several programming and insight exercises/questions. 
These exercises are meant to let you practice with the theory covered in: [Chapter 10][2] from "Mining of Massive Datasets" by J. Leskovec, A. Rajaraman, J. D. Ullman. <br>

**WHY** Practicing, both through programming and answering the insight questions, aims at deepening your knowledge and preparing you for the exam. <br>

**HOW** Follow the exercises in this notebook either on your own or with a friend. Use Mattermost
to disscus questions with your peers. For additional questions and feedback please consult the TA's at the assigned lab session. The answers to these exercises will not be provided.

[1]: https://mattermost.ewi.tudelft.nl/signup_user_complete/?id=ccffzw3cdjrkxkksq79qbxww7a
[2]: http://infolab.stanford.edu/~ullman/mmds/ch10.pdf

In this exercise, we will practice with Spectral Clustering to analyse Social Networks. To this end, we will create an adjacency matrix, set up a Laplacian matrix, compute the eigenvalue decomposition and perform clustering. Finally, we will use the code that you develop to cluster a large Social Network graph into more than two clusters.

**NOTICE: the aim of this lab is to give you deeper understanding of spectral clustering. To that end, it will require you to do solve some mathematical equations and dust off your linear algebra skills. So take out your pen and paper and be ready to write out the math related parts.**

# Spectral Graph Clustering

You are working for a popular social networking site, FriendBase. Your managers have thought of a wonderful new feature: InstaGroups<sup>tm</sup>. InstaGroups<sup>tm</sup> will automatically suggest a clustering of your group of friends, so you can easily send messages to- and post memes meant for only a select group.

In order to start working on this problem, you are given a small dataset with nine people (given as a list) and their friendships. The friendships are provided in a list of tuples, where each tuple in the list represents a friendships, e.g.: `('Albert', 'Bob')` represents a friendship between Albert and Bob. Each friendship is undirected, so a friendship defined for `('Albert', 'Bob')` will also hold for `('Bob', 'Albert')`.

In [ ]:
class graph(object):
    """
    Very simple graph class that holds a list of nodes and a list of edges connecting the nodes.
    Feel free to extend the functionality of this class to better organise your code.
    """
    
    def __init__(self, nodes=[], edges=[]):
        self.nodes = nodes
        self.edges = edges

In [ ]:
# List of people, the nodes in the graph.
people = ['Alice',
           'Bob',
           'Claudia',
           'Dennis',
           'Ellie',
           'Frida',
           'George',
           'Harry',
           'Irene']

# Friendships between people, the edges in the graph.
friendships = [('Alice', 'Bob'),
               ('Alice', 'Claudia'),
               ('Alice', 'Dennis'),
               ('Bob', 'Claudia'),
               ('Bob', 'Dennis'),
               ('Bob', 'Frida'),
               ('Claudia', 'Dennis'),
               ('Claudia', 'Irene'),
               ('Dennis', 'Ellie'),
               ('Ellie', 'Frida'),
               ('Ellie', 'George'),
               ('Frida', 'George'),
               ('Frida', 'Harry'),
               ('George', 'Harry')]

friend_graph = graph(people, friendships)

### Question 1

Draw a graph of this network for yourself to visualise the network.
- What are the nodes? What are the edges in your graph?
- How many clusters would you create? Which cluster would each person be assigned to?

### Exercise 1 - The Adjacency Matrix

We will now apply Spectral Clustering to this problem. To do so, we will need an adjacency matrix of this social network.

Remember that the adjacency matrix $\mathbf{A}$ is an $n \times n$ matrix, where $n$ is the number of nodes in your graph. The entry at row $i$ and column $j$ is 1 if there is an edge between node $i$ and node $j$. We denote this as $a_{ij} = 1$, otherwise, $a_{ij} = 0$.

Construct the adjacency matrix for the provided dataset.

Hint: you can use `list.index(element)` to find the index of a given element.

In [ ]:
import numpy as np

def create_adjacency_matrix(graph):
    """
    Creates and returns the adjacency matrix for a given graph.
    """
    
    adjacency_matrix = np.zeros((len(graph.nodes), len(graph.nodes)))
    
    # Start coding here!
    
    return adjacency_matrix

adjacency_matrix = create_adjacency_matrix(friend_graph)

adjacency_matrix

### Exercise 2 - Graph Laplacian

The next step is to compute the Laplacian matrix given this adjacency matrix. The Laplacian of a graph is defined as

$L = D - A$

$D$ is the degree matrix. The degree matrix describes the number of edges for each node on the diagonal of an $n \times n$ matrix:<br>
$d_{ii} = \sum_{j \in \delta(i)} 1$.

Complete the provided functions to compute the Laplacian of the graph for the FriendBook dataset.

In [ ]:
def compute_degree_matrix(adjacency_matrix):
    """
    Computes the degree matrix from an adjacency matrix.
    """
    
    degree_matrix = np.zeros_like(adjacency_matrix)
    
    # Start coding here!
    
    return degree_matrix

def compute_laplacian(adjacency_matrix):
    """
    Computes the Laplacian matrix from an adjacency matix.
    """
    
    laplacian = np.zeros_like(adjacency_matrix)
    
    # Start coding here!
    
    return laplacian

laplacian = compute_laplacian(adjacency_matrix)

laplacian

### Question 2

In order to grow your intuition of the Laplacian matrix, we will do a small exercise. Let's say you know the height of every person in the dataset. You can store these heights in a vector $\mathbf{v}$ of length $n$. The $i$<sup>th</sup> element of the vector, $v_i$, stores the height for person $i$.

- What happens if you multiply the Laplacian matrix with the heights of each person, i.e. $\mathbf{L}\mathbf{v}$? Try doing this with a very small graph consiting of three nodes: a, b, c, where (a, b) and (a, c) are connected and the heights of a, b, and c are 2, 3, and 4, respectively.  
  
- What if all people have the same height, e.g. $\mathbf{v} = \mathbf{1} = [1, 1, 1, ...]^T$?  
  
- What are you computing this way? Hint: what's the difference in height between each pair? How would you compute the average of these differences?  

### Exercise 3 - Eigenvalue decomposition

The next step in the Spectral Clustering algorithm is to compute the Eigenvalue decomposition of the Laplacian matrix. If you would like to better understand Eigenvalues and Eigenvectors, watch [this video on eigenvectors][1].

Compute the eigenvalue decomposition of the Laplacian matrix and print the eigenvalues and eigenvectors corresponding to the *first three* eigenvalues in increasing order.

Hint 1: Use numpy's [linear algebra library][2].<br>
Hint 2: Use `np.argsort(list)` to get the sorted indices of a list.

[1]: https://www.youtube.com/watch?v=PFDu9oVAE-g&list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab&index=13
[2]: https://docs.scipy.org/doc/numpy/reference/routines.linalg.html

In [ ]:
from numpy import linalg as la

"""
Numpy's Linear Algebra returns a vector of eigenvalues and a matrix
where the columns are the corresponding eigenvectors.
The eigenvalues are not necessarily ordered.
"""
v, w = la.eig(laplacian)

# Start coding here!

### Question 3
What is the first eigenvalue and its corresponding eigenvector?  Did you expect this result?  

### Exercise 4
You will now perform the final step in the Spectral Clustering algorithm: the actual clustering. You will do this using the second eigenvector of the Laplacian matrix.

Each person is assigned to a cluster based on the sign of their entry in the eigenvector. For example: if we have eigenvector $[3, 7, -2, 4, -3, 5, 2, -8, 1]^T$, we know that Alice, Bob, Dennis, Frida, George, and Irene should be in one cluster and that Claudia, Ellie, and Harry are in the other.

Complete the following functions to create and print the cluster assignment in a readable way.

In [ ]:
def second_eigenvector(laplacian):
    v, w = la.eig(laplacian)
    
    second_v = np.ones((laplacian.shape[0], 1))
    
    # Start coding here!
    
    return second_v

def spectral_cluster(g):
    """
    Clusters a graph given by nodes and edges using spectral clustering.
    Returns two graphs given by nodes1, edges1 and nodes2, edges2, respectively.
    """
    nodes1, edges1 = [], []
    nodes2, edges2 = [], []
    
    adjacency_matrix = create_adjacency_matrix(g)
    laplacian = compute_laplacian(adjacency_matrix)
    
    second_v = second_eigenvector(laplacian)
  
    # Start coding here!
    
    g1 = graph(nodes1, edges1)
    g2 = graph(nodes2, edges2)
    
    return g1, g2
        
graph1, graph2 = spectral_cluster(friend_graph)
print("Cluster 1:", graph1.nodes)
print("Cluster 2:", graph2.nodes)

### Question 4
Compare the results to your answer to question 1.1. Did you get the same clustering?  

### Exercise 5.a
Now we will extend this method to create partitions greater than $2$. As described in the lecture, there are two possible ways to proceed:
1. Recursively partition each cluster using the spectral clustering algorithm until you have reached $k$ partitions.
2. Use $d$ eigenvectors to construct a $d$-dimensional space and apply a classical clustering algorithm.

The first technique is quite straightforward, so let's implement it right away and see how it performs. For now, we will limit our implementation to $k$ being powers of $2$.

In [ ]:
from math import log2

def recursive_cluster_k(g, k):
    assert k <= len(g.nodes)
    assert log2(k) % 1 == 0
    
    depth = log2(k)
    return recursive_cluster(g, depth)

def recursive_cluster(g, depth):
    """
    Recursively clusters graph g until depth is 0.
    If you want, you can also implement this method iteratively (with a for loop).
    """
    # base case
    if depth == 0:
        return [g]
     
    clusters = []
    
    # Start coding here!

    return clusters
        
clusters = recursive_cluster_k(friend_graph, 4)
for cluster in clusters:
    print(cluster.nodes)
        

### Exercise 5.b

Now for the second technique: Use $d$ eigenvectors to construct a $d$-dimensional space and apply a classical clustering algorithm. Let's break up this sentence to see what it means.

1. _Use $d$ eigenvectors to construct a $d$-dimensional space._

The $d$ eigenvectors you select are the bases of a $d$-dimensional space (check out the video linked in exercise 3 to see why). In order to get the coördinates of a point in this new space, all we need to do is compute the dot product between the vector representation of that point in the original space and the eigenvector corresponding to each dimension. The vector representation $\mathbf{v}$ of a node $i$ in the original space is an $n \times 1$ vector with all zeros and $v_i = 1$.

If compute the dot product with each eigenvector, you get $d$ 'coördinates' for that node in the new $d$-dimensional space, which is called the spectral transform.

We can use a quick shortcut to get the coördinates for each node in the graph: we can simply concatenate the $d$ eigenvectors into an $n \times d$ matrix (try writing it out it out). The dataset of new features will look like this: [$w_1$, $w_2$, ..., $w_d$] where $w_j$ is the $j$-th eigenvector.

2. _Apply a classical clustering algorithm._

Now that you have $d$ coördinates for each node, we can feed these new coördinates (or features) into a standard clustering algorithm, like hierarchical clustering or k-means. We will not implement this step. For now, it will do to plot the new coordinates. Of course, you are free to use an out-of-the-box implementation or your own clustering implementation from the Machine Learning course.

*Let's get started!* We will use only two eigenvectors, so we can plot the new coordinates. Remember that we don't use the very first eigenvector, as it consists of only 1s.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

def spectral_coordinates(laplacian, d):
    v, w = la.eig(laplacian)
    
    coordinates = np.ones((laplacian.shape[0], d))
    
    # Start coding here!

    return coordinates

def spectral_cluster_k(g, k):    
    adjacency_matrix = create_adjacency_matrix(g)
    laplacian = compute_laplacian(adjacency_matrix)
    
    coordinates = spectral_coordinates(laplacian, 2)
    
    for i in range(len(coordinates)):
        plt.scatter(coordinates[i, 0], coordinates[i, 1])
    plt.legend(people)
    plt.show()

spectral_cluster_k(friend_graph, 4)

### Question 5
- What do the axes mean in this plot? 
- Would you get the same clustering results using the two methods? What is the difference?